In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import pandas as pd
import shutil
import time
import csv
import os

### Coversione da .txt/.tab a Dataframe
* creo un DataFrame vuoto
* quando raggiungo la riga dei nomi delle colonne riporto gli stessi nomi per le colonne del DataFrame
* aggiungo ogni riga trovata al dataframe

In [2]:
#--- FUNZIONE PER CONVERTIRE UNA TABELLA IN DATAFRAME

def tab_to_df(filename):
        
    file1 = open(filename, "rb")
    df = pd.DataFrame()
    flag_data = 0
    
    #pbar = tqdm(file1, desc=filename)
    for line in tqdm(file1):

        if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
            pass
        else:
            line = line.decode() # altrimenti decodifico la riga

            # cerco la riga con i nomi della tabella
            if line.startswith('NR'):
                flag_data = 1                  # quando trovo i nomi delle colonne imposto flag_data = 1
                table_names = line.split()     # divido i nomi delle colonne      
                table_names.remove('LastLine') # elimino 'LastLine' che fa riferimento ad un singolo dato duplicato
                
                # definisco i nomi delle colonne del dataframe
                df = df.reindex(columns = table_names)

            # Se sono arrivato alla tabella ma la riga inizia con 0 la salto
            elif flag_data == 1 and line.startswith('0'):
                pass

            # Se sono arrivato alla tabella (flag_data == 1) e non sono alla prima riga
            # inizio a salvare i dati
            elif flag_data == 1:
                row = line.split()

                if not line.startswith('[END]') and len(row) == len(table_names)+1: # le righe con meno di 25 elementi hanno dati mancanti
                    row[-2] = str(row[-2]) + ' ' + str(row[-1]) # unisco l'ora col giorno
                    del row[-1] # elimino la colonna con i giorni
                    
                    df_length = len(df)
                    df.loc[df_length] = row
    
    #df['DataTime'] = pd.to_datetime(df['DataTime'], format='%H:%M:%S %d.%m.%y', errors='coerce')
    df.set_index('DataTime', drop = True, inplace=True)
    df.index = pd.to_datetime(df.index, format='%H:%M:%S %d.%m.%Y')
    df.drop('NR', axis=1, inplace=True)
    
    for column in df.keys():
        if 'PLC_Byt' in column or column == 'OpMode':
            df.drop(column, axis=1, inplace=True)
        else:
            df[column] = df[column].apply(pd.to_numeric)
    return df
                        
#--- MAIN

#data = tab_to_df('table_new/Diag_temp.tab')
#data.head(50)

In [ ]:
path = 'table_new2/'
path_dest = path + 'csv/'
dataDict = {}

# controllo se la cartella 'csv' è presente. Se c'è la elimino e la ricreo, se non c'è la creo.
if os.path.isdir(path_dest):
    shutil.rmtree(path_dest)
    os.mkdir(path_dest)
else:
    os.mkdir(path_dest)

for filename in os.listdir(path):
    if (filename.endswith('.txt') or filename.endswith('.tab')): 
        df = tab_to_df(path + filename)
        if len(df) > 5:
            #df = df.resample('10T').mean()
            df.sort_index(inplace = True)
            dataDict[filename] = df
            df.to_csv(path_dest+filename, index=True)

for i in dataDict:
    print(i)

In [3]:
path = 'table_new2/csv/'
dataDict = {}
for filename in os.listdir(path):
    df = pd.read_csv(path + filename)
    df['DataTime'] = pd.to_datetime(df['DataTime'], format='%Y-%m-%d %H:%M:%S')
    df.set_index('DataTime', drop = True, inplace=True)
    dataDict[filename] = df
    
for i in dataDict:
    print(i)

Long_term_temp.tab
DomoClima_temp.tab
Diag_temp.tab
DiagnosisProfibus.tab
Diag_temp.txt
DiagnosisM398.tab
hour_air_consumption.tab


In [ ]:
dataDict['Long_term_temp.tab'].head()

In [ ]:
for tab in dataDict:
    df = dataDict[tab].resample('1H').mean()
    df.plot(figsize=(15,10), title=tab)

In [ ]:
df = dataDict['hour_air_consumption.tab']
df = df.reset_index()

df['DataTime'] = df['DataTime'].apply(lambda x: x.replace(second=0))
df = df.drop_duplicates(subset='DataTime', keep='last')
df = df.set_index('DataTime')
df.sort_index(inplace = True)
#df['2021-07-07':].plot(figsize=(15,10))
df = df.resample('1H').mean()
fig = px.line(df, height=800)
fig.show()

In [21]:
from datetime import date

max_date_allowed=date.today()
print(type(max_date_allowed))
print(max_date_allowed)

#dataDict['Diag_temp.tab'].info()

date_list= pd.DatetimeIndex(dataDict['Diag_temp.tab'].index)
print(date_list[0])

<class 'datetime.date'>
2021-07-20
2021-07-11 08:25:01
